In [2]:
import time
import toml
import numpy as np
import matplotlib.pyplot as plt

from typing import Dict, Optional
from binance.client import Client
from datetime import datetime, timezone

from filter_conform import get_exchange_info_for_symbol

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 18})

In [3]:
cfg = toml.load("configuration.toml")
pkey = cfg["auth"]["pkey"]
skey = cfg["auth"]["skey"]

# client = Client(pkey, skey)  ## commented so i dont accidentally buy myself doge again lol

## How to buy/sell a coin

Before anything else, the order has to conform to order filters, which can be retrieved 

In [48]:
from binance.client import Client

get_exchange_info_for_symbol(client, "DOGEUSDT")

{'symbol': 'DOGEUSDT',
 'status': 'TRADING',
 'baseAsset': 'DOGE',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00001000',
   'maxPrice': '1000.00000000',
   'tickSize': '0.00001000'},
  {'filterType': 'PERCENT_PRICE',
   'multiplierUp': '5',
   'multiplierDown': '0.2',
   'avgPriceMins': 5},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.10000000',
   'maxQty': '9000000.00000000',
   'stepSize': '0.10000000'},
  {'filterType': 'MIN_NOTIONAL',
   'minNotional': '10.00000000',
   'applyToMarket': True,
   'avgPriceMins': 5},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},


Important things in the exchange info:
- baseAssetPrecision / quoteAssetPrecision - neet to format orders to that precision
- filters - must conform order to filters

For constructing orders: I assume we have the desired quantity of USD / crypto that we want to buy or sell. For simplicity, take this to be a limit order at first. We can check that the order would pass the filters before sending it to binance. This would allow us to, in the future, modify orders before sending them in.

In [41]:
client.get_ticker(symbol="DOGEBTC")

{'symbol': 'DOGEBTC',
 'priceChange': '0.00000124',
 'priceChangePercent': '11.912',
 'weightedAvgPrice': '0.00001065',
 'prevClosePrice': '0.00001041',
 'lastPrice': '0.00001165',
 'lastQty': '345.00000000',
 'bidPrice': '0.00001165',
 'bidQty': '1556.00000000',
 'askPrice': '0.00001166',
 'askQty': '90454.00000000',
 'openPrice': '0.00001041',
 'highPrice': '0.00001196',
 'lowPrice': '0.00000921',
 'volume': '1700221573.00000000',
 'quoteVolume': '18111.16549559',
 'openTime': 1620337565561,
 'closeTime': 1620423965561,
 'firstId': 28137804,
 'lastId': 28925954,
 'count': 788151}

In [45]:
client.order_market_sell(
    symbol="DOGEBTC",
    quantity=9.
)

{'symbol': 'DOGEBTC',
 'orderId': 150062212,
 'orderListId': -1,
 'clientOrderId': 'l0Ky2xOtwYiuBROjp2yw03',
 'transactTime': 1620424070008,
 'price': '0.00000000',
 'origQty': '9.00000000',
 'executedQty': '9.00000000',
 'cummulativeQuoteQty': '0.00010584',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'fills': [{'price': '0.00001176',
   'qty': '9.00000000',
   'commission': '0.00000011',
   'commissionAsset': 'BTC',
   'tradeId': 28928715}]}

In [46]:
client.get_trade_fee(symbol='DOGEBTC')

{'success': True,
 'tradeFee': [{'maker': 0.001, 'symbol': 'DOGEBTC', 'taker': 0.001}]}